In [1]:
train = read.csv('../data/train.csv')
test = read.csv('../data/test.csv')

In [2]:
summary(train)

       ID          YearBuilt         SqFt          Story      
 Min.   : 1.00   Min.   :1909   Min.   : 705   Min.   :1.000  
 1st Qu.:23.25   1st Qu.:1966   1st Qu.:1224   1st Qu.:1.000  
 Median :45.50   Median :1981   Median :1590   Median :1.000  
 Mean   :45.50   Mean   :1976   Mean   :1747   Mean   :1.361  
 3rd Qu.:67.75   3rd Qu.:1987   3rd Qu.:2088   3rd Qu.:1.750  
 Max.   :90.00   Max.   :2003   Max.   :4650   Max.   :2.500  
                                                              
     Acres             Baths         Fireplaces          Zip       
 Min.   : 0.0000   Min.   :1.000   Min.   :0.0000   Min.   :27502  
 1st Qu.: 0.2125   1st Qu.:2.000   1st Qu.:0.0000   1st Qu.:27529  
 Median : 0.3250   Median :2.000   Median :1.0000   Median :27604  
 Mean   : 0.8513   Mean   :2.022   Mean   :0.7333   Mean   :27576  
 3rd Qu.: 0.5450   3rd Qu.:2.500   3rd Qu.:1.0000   3rd Qu.:27610  
 Max.   :39.3800   Max.   :3.000   Max.   :2.0000   Max.   :27617  
                   N